## Prepare Training Dataset

In [1]:
!pip install easydict

In [2]:
import easydict

In [3]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
from itertools import chain
from sklearn.model_selection import train_test_split

from pprint import pprint


class CustomDataset(Dataset):
    def __init__(self, args, data, mode):
        self.data = data
        self.data_dir = args.data_dir
        self.mode = mode
        self.tokenizer = AutoTokenizer.from_pretrained(args.model_name)
        self.inputs, self.labels = self.data_loader()

    def data_loader(self):
        print('Loading ' + self.mode + ' dataset..')
        if os.path.isfile(os.path.join(self.data_dir, self.mode + '_X.pt')):
            inputs = torch.load(os.path.join(self.data_dir, self.mode + '_X.pt'))
            labels = torch.load(os.path.join(self.data_dir, self.mode + '_Y.pt'))

        else:
            df = self.data
            inputs = pd.DataFrame(columns=['src'])
            labels = pd.DataFrame(columns=['trg'])
            inputs['src'] =  df['article_original']

            if self.mode != "test":
                labels['trg'] =  df['extractive']

            # Preprocessing
            inputs, labels = self.preprocessing(inputs, labels)
            print("preprocessing")

            # Save data
            torch.save(inputs, os.path.join(self.data_dir, self.mode + '_X.pt'))
            torch.save(labels, os.path.join(self.data_dir, self.mode + '_Y.pt'))

        inputs = inputs.values
        labels = labels.values

        return inputs, labels

    def pad(self, data, pad_id, max_len):
        padded_data = data.map(lambda x : torch.cat([x, torch.tensor([pad_id] * (max_len - len(x)), dtype=torch.int64)]))
        return padded_data
    
    def tokenize(self, x):
        result = [self.tokenizer.encode(x[i], add_special_tokens=True) for i in range(len(x))]
        result_concat = list(chain.from_iterable(result))
        
        if len(result_concat) <= 512:
            return torch.tensor(result_concat)
            
        else:
            length_sum = 0
            for sen_token in result:
                length_sum += len(sen_token)
                
            return torch.tensor(list(chain.from_iterable([self.tokenizer.encode(x[i], max_length = int(512 / len(x)), add_special_tokens=True) for i in range(len(x))])))


    def preprocessing(self, inputs, labels):
        print('Preprocessing ' + self.mode + ' dataset..')

        # Encoding original text
        inputs['src'] = inputs['src'].map(self.tokenize)
        # inputs['src'] = inputs['src'].map(lambda x: torch.tensor(list(chain.from_iterable([self.tokenizer.encode(x[i], max_length = int(512 / len(x)), add_special_tokens=True) for i in range(len(x))]))))
        inputs['clss'] = inputs.src.map(lambda x : torch.cat([torch.where(x == 2)[0], torch.tensor([len(x)])]))
        inputs['segs'] = inputs.clss.map(lambda x : torch.tensor(list(chain.from_iterable([[0] * (x[i+1] - x[i]) if i % 2 == 0 else [1] * (x[i+1] - x[i]) for i, val in enumerate(x[:-1])]))))
        inputs['clss'] = inputs.clss.map(lambda x : x[:-1])
        
        # Padding
        max_encoding_len = max(inputs.src.map(lambda x: len(x)))
        max_label_len = max(inputs.clss.map(lambda x: len(x)))
        inputs['src'] = self.pad(inputs.src, 0, max_encoding_len)
        inputs['segs'] = self.pad(inputs.segs, 0, max_encoding_len)
        inputs['clss'] = self.pad(inputs.clss, -1, max_label_len)
        inputs['mask'] = inputs.src.map(lambda x: ~ (x == 0))
        inputs['mask_clss'] = inputs.clss.map(lambda x: ~ (x == -1))

        # Binarize label {Extracted sentence : 1, Not Extracted sentence : 0}

        if self.mode != 'test':
            labels = labels['trg'].map(lambda  x: torch.tensor([1 if i in x else 0 for i in range(max_label_len)]))

        return inputs, labels


    def __len__(self):
        return len(self.inputs)


    def __getitem__(self, index):
        if self.mode == 'test':
            return [self.inputs[index][i] for i in range(5)]
        else:
            return [self.inputs[index][i] for i in range(5)], self.labels[index]


def get_train_loaders(args):
    """
        define train/validation pytorch dataset & loader

        Returns:
            train_loader: pytorch data loader for train data
            val_loader: pytorch data loader for validation data
    """
    # get data from json
    with open(os.path.join(args.data_dir, "train.json"), "r", encoding="utf-8-sig") as f:
        data = pd.read_json(f) 
    train_df = pd.DataFrame(data)
    
    if args.train_kfold:
        kf = KFold(n_splits=5)
        for fold, (train_idx, val_idx) in enumerate(kf.split(train_df)):
            if args.fold != fold:
                continue
            train_data = train_df.iloc[train_idx]
            val_data = train_df.iloc[val_idx]        
    else:
        train_data, val_data = train_test_split(train_df, test_size=0.1, random_state=args.seed)
    
    # get train & valid dataset from dataset.py
    train_dataset = CustomDataset(args, train_data, mode='train')
    val_dataset = CustomDataset(args, val_data, mode='valid')

    # define data loader based on each dataset
    train_dataloader = DataLoader(dataset=train_dataset,
                                  batch_size=args.batch_size,
                                  num_workers=args.num_workers,
                                  pin_memory=True,
                                  drop_last=False,
                                  shuffle=True)
    val_dataloader = DataLoader(dataset=val_dataset,
                                batch_size=args.batch_size,
                                num_workers=args.num_workers,
                                pin_memory=True,
                                drop_last=False,
                                shuffle=False)

    return train_dataloader, val_dataloader

In [4]:
config = {}

# 설정
config['seed'] = 981201
config['device'] = "cuda" if torch.cuda.is_available() else "cpu"
config['data_dir'] = '/opt/ml/Legal-Document-Summarization/data'
config['model_name'] = 'klue/bert-base'
config['batch_size'] = 32
config['num_workers']= 4
config['train_kfold']= False


args = easydict.EasyDict(config)

In [5]:
with open(os.path.join(args.data_dir, "train.json"), "r", encoding="utf-8-sig") as f:
        data = pd.read_json(f) 
train_df = pd.DataFrame(data)

In [6]:
train_df.head()

,id,extractive,article_original
0,196524,"[13, 14, 15]","[[1] 의용 부동산등기법 제35조 제1항,, 제60조 제1항은 등기를 신청함에는 ..."
1,106984,"[1, 2, 4]",[가. 주택개량을 위하여 조합원들이 스스로 결성한 주택개량재개발조합이 실시하는 재개...
2,190919,"[1, 2, 3]","[금원의 목적 내지 성질상 국가나 지방자치단체와 특정인 사이에서만 수수,, 결제되어..."
3,110573,"[0, 2, 5]",[가. 자동차대여업자의 직원으로서는 운전면허 없는 운전자가 위조된 운전면허증의 복사...
4,156698,"[0, 2, 3]",[소외회사의 평리사 6명을 제쳐 놓고 대표이사 3명만의 결의에 의하여 동회사의 대표...


In [7]:
inputs = pd.DataFrame(columns=['src'])
labels = pd.DataFrame(columns=['trg'])
inputs['src'] =  train_df['article_original']
labels['trg'] =  train_df['extractive']

In [8]:
inputs.head()

,src
0,"[[1] 의용 부동산등기법 제35조 제1항,, 제60조 제1항은 등기를 신청함에는 ..."
1,[가. 주택개량을 위하여 조합원들이 스스로 결성한 주택개량재개발조합이 실시하는 재개...
2,"[금원의 목적 내지 성질상 국가나 지방자치단체와 특정인 사이에서만 수수,, 결제되어..."
3,[가. 자동차대여업자의 직원으로서는 운전면허 없는 운전자가 위조된 운전면허증의 복사...
4,[소외회사의 평리사 6명을 제쳐 놓고 대표이사 3명만의 결의에 의하여 동회사의 대표...


In [9]:
inputs.iloc[0][0]

['[1] 의용 부동산등기법 제35조 제1항,',
 '제60조 제1항은 등기를 신청함에는 등기원인을 증명하는 서면을 제출하여야 하고,',
 '등기관리가 등기를 완료한 때에는 등기원인을 증명하는 서면',
 '또는 신청서 부본에 등기번호,',
 '신청서 수부(受附) 연월일,',
 '수부번호(受附番號),',
 '순위번호 및 등기제(登記濟)의 뜻을 기재하고,',
 '등기소의 인을 압날하여 이를 등기권리자에게 환부하여야 한다고 규정하고 있다.',
 '그러므로 매도증서에 위 규정에 따른 등기번호,',
 '등기순위, 등기제 등의 기재와 등기소인이 날인되어 있는 사실이 인정된다면,',
 '이는 등기 신청 시 등기원인을 증명하는 서면으로 제출되었다가 등기관리가 등기를 완료하고',
 '등기권리자에게 되돌려준 것으로 보지 않을 수 없다.',
 '따라서 특별한 사정이 없는 한 그 서면에 기재된 부동산에 관하여 그 기재의 등기번호와 순위번호에 따른 등기가 마쳐졌다고 인정하여야 한다.',
 '[2] 민사소송법 제356조 제1항은 문서의 작성방식과 취지에 의하여 공무원이 직무상 작성한 것으로 인정한 때에는 이를 진정한 공문서로 추정한다고 규정하고 있으나,',
 '위조 또는 변조 등 특별한 사정이 있다고 볼 만한 반증이 있는 경우에는 위와 같은 추정은 깨어진다.',
 '[3] 매도증서 등에 등기소의 등기제(登記濟)의 기재가 첨가됨으로써 사문서와 공문서로 구성된 문서는 공증에 관한 문서와는 달라 공문서 부분 성립이 인정된다고 하여 바로 사문서 부분인 매도증서 자체의 진정성립이 추정되거나 인정될 수는 없다.']

In [10]:
labels.head()

,trg
0,"[13, 14, 15]"
1,"[1, 2, 4]"
2,"[1, 2, 3]"
3,"[0, 2, 5]"
4,"[0, 2, 3]"


## Make word index dictionary

In [11]:
tokenizer = AutoTokenizer.from_pretrained(args.model_name)

In [12]:
def tokenize(x):
    result = [tokenizer.encode(x[i], add_special_tokens=True) for i in range(len(x))]
    result_concat = list(chain.from_iterable(result))

    if len(result_concat) <= 512:
        return torch.tensor(result_concat)
    
    else:
        length_sum = 0
        for sen_token in result:
            length_sum += len(sen_token)

        return torch.tensor(list(chain.from_iterable([tokenizer.encode(x[i], max_length = int(512 * (len(result[i]) / length_sum)-1), add_special_tokens=True, truncation=True) for i in range(len(x))])))

In [13]:
inputs['src'] = inputs['src'].map(tokenize)

In [14]:
print(tokenizer.convert_ids_to_tokens(inputs['src'][0]))

['[CLS]', '[', '1', ']', '의', '##용', '부동산', '##등기', '##법', '제', '##35', '##조', '제', '##1', '##항', ',', '[SEP]', '[CLS]', '제', '##60', '##조', '제', '##1', '##항', '##은', '등기', '##를', '신청', '##함', '##에', '##는', '등기', '##원', '##인', '##을', '증명', '##하', '##는', '서면', '##을', '제출', '##하여', '##야', '하고', ',', '[SEP]', '[CLS]', '등기', '##관리', '##가', '등기', '##를', '완료', '##한', '때', '##에', '##는', '등기', '##원', '##인', '##을', '증명', '##하', '##는', '서면', '[SEP]', '[CLS]', '또는', '신청서', '부', '##본', '##에', '등기', '##번', '##호', ',', '[SEP]', '[CLS]', '신청서', '수부', '(', '受', '[UNK]', ')', '연', '##월일', ',', '[SEP]', '[CLS]', '수부', '##번', '##호', '(', '受', '[UNK]', '[UNK]', '[UNK]', ')', ',', '[SEP]', '[CLS]', '순위', '##번', '##호', '및', '등기', '##제', '(', '[UNK]', '記', '濟', ')', '의', '뜻', '##을', '기재', '##하고', ',', '[SEP]', '[CLS]', '등기', '##소', '##의', '인', '##을', '압', '##날', '##하여', '이를', '등기', '##권', '##리', '##자', '##에', '##게', '환', '##부', '##하여', '##야', '한다고', '규정', '##하고', '있', '##다', '.', '[SEP]', '[CLS]', '그러므로', '매

In [15]:
def make_word_index_dict(tokens):
    word_index = {}
    word = ''
    index = []
    
    for i, t in enumerate(tokens):
        if (t == '[CLS]') or (t == '[SEP]'):
            continue
        if not t.startswith('##'):
            if word:
                word_index[word.replace('##', '')] = index
                word = ''
                index = []
            word += t
            index.append(i)
        if t.startswith('##'):
            word += t
            index.append(i)
                
    return word_index

In [16]:
for src in inputs['src']:
    tokens = tokenizer.convert_ids_to_tokens(src)
    word_dict = make_word_index_dict(tokens)
    print(word_dict, end='\n\n')
    sorted_dict = [key for key in list(word_dict.keys()) if len(key) > 2]
    print(sorted_dict)
    
    break

{'[': [357], '1': [2], ']': [359], '의용': [4, 5], '부동산등기법': [6, 7, 8], '제35조': [9, 10, 11], '제1항': [12, 13, 14], ',': [324], '제60조': [18, 19, 20], '제1항은': [287, 288, 289, 290], '등기를': [210, 211], '신청함에는': [27, 28, 29, 30], '등기원인을': [194, 195, 196, 197], '증명하는': [198, 199, 200], '서면을': [38, 39], '제출하여야': [40, 41, 42], '하고': [43], '등기관리가': [207, 208, 209], '완료한': [52, 53], '때에는': [310, 311, 312], '서면': [64], '또는': [328], '신청서': [78], '부본에': [69, 70, 71], '등기번호': [156, 157, 158], '수부': [79], '(': [370], '受': [93], '[UNK]': [371], ')': [374], '연월일': [84, 85], '수부번호': [89, 90, 91], '순위번호': [101, 102, 103], '및': [104], '등기제': [368, 369], '記': [372], '濟': [373], '의': [375], '뜻을': [113, 114], '기재하고': [115, 116], '등기소의': [365, 366, 367], '인을': [123, 124], '압날하여': [125, 126, 127], '이를': [313], '등기권리자에게': [216, 217, 218, 219, 220, 221], '환부하여야': [135, 136, 137, 138], '한다고': [139], '규정하고': [320, 321], '있다': [142, 143], '.': [354], '그러므로': [147], '매도증서에': [148, 149, 150, 151], '위': [152], '규정에': [15

## Masking to random tokens

In [17]:
import random

In [18]:
def mask_to_random_tokens(inputs, k=2):
    mask_token_id = tokenizer.mask_token_id
    
    for src in inputs:
        index_to_mask = []
        
        tokens = tokenizer.convert_ids_to_tokens(src)
        word_dict = make_word_index_dict(tokens)
        candidates = [key for key in list(word_dict.keys()) if len(key) > 2]
        
        for i in range(k):
            rand_num = random.randint(0, len(candidates)-1)
            index_to_mask.extend(word_dict[candidates[rand_num]])
            
        for idx in index_to_mask:
            src[idx] = mask_token_id
        
    return inputs

In [19]:
def mask_to_long_tokens(inputs, k=2):
    mask_token_id = tokenizer.mask_token_id
    
    for src in inputs:
        index_to_mask = []
        
        tokens = tokenizer.convert_ids_to_tokens(src)
        word_dict = make_word_index_dict(tokens)
        candidates = sorted(list(word_dict.keys()), reverse=True, key=len)
        
        for i in range(k):
            index_to_mask.extend(word_dict[candidates[i]])
            
        for idx in index_to_mask:
            src[idx] = mask_token_id
        
    return inputs

In [20]:
%%time
inputs['src'] = mask_to_long_tokens(inputs['src'])

CPU times: user 39.3 s, sys: 0 ns, total: 39.3 s
Wall time: 39.3 s


In [22]:
print(tokenizer.convert_ids_to_tokens(inputs['src'][0]))

['[CLS]', '[', '1', ']', '의', '##용', '[MASK]', '[MASK]', '[MASK]', '제', '##35', '##조', '제', '##1', '##항', ',', '[SEP]', '[CLS]', '제', '##60', '##조', '제', '##1', '##항', '##은', '등기', '##를', '신청', '##함', '##에', '##는', '등기', '##원', '##인', '##을', '증명', '##하', '##는', '서면', '##을', '제출', '##하여', '##야', '하고', ',', '[SEP]', '[CLS]', '등기', '##관리', '##가', '등기', '##를', '완료', '##한', '때', '##에', '##는', '등기', '##원', '##인', '##을', '증명', '##하', '##는', '서면', '[SEP]', '[CLS]', '또는', '신청서', '부', '##본', '##에', '등기', '##번', '##호', ',', '[SEP]', '[CLS]', '신청서', '수부', '(', '受', '[UNK]', ')', '연', '##월일', ',', '[SEP]', '[CLS]', '수부', '##번', '##호', '(', '受', '[UNK]', '[UNK]', '[UNK]', ')', ',', '[SEP]', '[CLS]', '순위', '##번', '##호', '및', '등기', '##제', '(', '[UNK]', '記', '濟', ')', '의', '뜻', '##을', '기재', '##하고', ',', '[SEP]', '[CLS]', '등기', '##소', '##의', '인', '##을', '압', '##날', '##하여', '이를', '등기', '##권', '##리', '##자', '##에', '##게', '환', '##부', '##하여', '##야', '한다고', '규정', '##하고', '있', '##다', '.', '[SEP]', '[CLS]', '그